In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
# 回傳opgg上某召喚師ID的最近二十場內有效牌位對戰英雄及結果(左上,左野,左中,左下,左輔,右上,右野,右中,右下,右輔,0=左輸1=左贏)
def get_match(region:int, summoner_id:str)->pd.DataFrame:
    soup = BeautifulSoup(requests.get(f'https://{["www", "na", "euw", "eune"][region]}.op.gg/summoner/userName={summoner_id.replace(" ", "+")}').text, 'html.parser')
    list_match = [i.text for i in soup.find_all("div", class_="Image16")]
    list_inf = [i.get_text(strip=True) for i in soup.select('div.GameSettingInfo > div.ChampionName')] + [i.get_text(strip=True) for i in soup.find_all("div", class_="GameType")] + [i.get_text(strip=True) for i in soup.find_all("div", class_="GameResult")]
    df = pd.DataFrame(np.append(np.reshape(list_match, (-1, 10)), np.reshape(list_inf, (3, -1)).T, 1))
    df = df[~df[12].str.contains("Remake") & df[11].str.contains("Ranked")]
    df['result'] = np.where((((df[0]==df[10])|(df[1]==df[10])|(df[2]==df[10])|(df[3]==df[10])|(df[4]==df[10]))&(df[12].str.contains("Victory")))|
                            (((df[5]==df[10])|(df[6]==df[10])|(df[7]==df[10])|(df[8]==df[10])|(df[9]==df[10]))&(df[12].str.contains("Defeat"))), 1, 0)
    df.drop(df.columns[[10, 11, 12]], axis = 1, inplace = True)
    return df